import libraries

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

load in data & id ideal species

In [2]:
from collections import Counter
from sklearn.preprocessing import LabelEncoder

# read in data
data = pd.read_csv('business_trees_1km_with_weather_clusters.csv')

# process to get most common tree species associated with business
data['NearbyTreeSpecies'] = data['NearbyTreeSpecies'].astype(str)

# Function to find the most common species
def get_most_common_species(nearby_species):
    if ', ' not in nearby_species:
        return nearby_species  # If there's only one species, return it
    species_list = nearby_species.split(', ')  # Split the string into a list of species
    species_counts = Counter(species_list)  # Count occurrences of each species
    most_common_species, _ = species_counts.most_common(1)[0]  # Get the most common species
    return most_common_species

# Apply the function to create a new column for the predicted species
data['BestTreeSpecies'] = data['NearbyTreeSpecies'].apply(get_most_common_species)

# Encode categorical features, including the target if it's categorical
encoder = LabelEncoder()
data['Category_encoded'] = encoder.fit_transform(data['Category'])
data['BestTreeSpecies_encoded'] = encoder.fit_transform(data['BestTreeSpecies'])

data = data.apply(pd.to_numeric, errors='coerce')
data = data.dropna(subset=['Total.SqFt', 'Category_encoded', 'Year.Built', 'kWh.sqft', 'lat', 'long', 'BestTreeSpecies_encoded'])

data.head()

,Unnamed: 0,Reported.Year,Property.ID..,Commercial.Property..Property.Street.Address,Building.Name,Total.SqFt,Category,Year.Built,kWh.sqft,Percentile.Rank,...,GeoLocation,Submitted.Name.of.Organization,GeoLocation_clean,lat,long,NearbyTreeSpecies,cluster,BestTreeSpecies,Category_encoded,BestTreeSpecies_encoded
0,1,2017.0,187746.0,NaN,NaN,7419.0,NaN,1940.0,0.000,NaN,...,NaN,NaN,NaN,30.251707,-97.697313,NaN,2,NaN,27,20
1,2,2017.0,230740.0,NaN,NaN,49024.0,NaN,1983.0,15.580,39.58,...,NaN,NaN,NaN,30.328069,-97.672401,NaN,2,NaN,10,18
2,3,2017.0,729528.0,NaN,NaN,101953.0,NaN,2009.0,32.063,76.26,...,NaN,NaN,NaN,30.236494,-97.721013,NaN,1,NaN,24,18
3,5,2017.0,526381.0,NaN,NaN,146142.0,NaN,1999.0,8.049,4.79,...,NaN,NaN,NaN,30.373732,-97.719236,NaN,2,NaN,16,18
4,6,2017.0,200001.0,NaN,NaN,15350.0,NaN,1998.0,100.129,NaN,...,NaN,NaN,NaN,30.279196,-97.740475,NaN,2,NaN,18,18


rando forest time

In [3]:
# Define features and target
features = data[['Total.SqFt', 'Category_encoded', 'Year.Built', 'kWh.sqft', 'lat', 'long']]
target = data['BestTreeSpecies_encoded']

# Convert to numeric for modeling purposes
features = features.apply(pd.to_numeric, errors='coerce')

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

# Initialize the Random Forest classifier
rf_clf = RandomForestClassifier(
    n_estimators=100,         # Number of trees in the forest
    max_depth=None,           # Maximum depth of each tree
    min_samples_split=2,      # Minimum samples required to split a node
    min_samples_leaf=1,       # Minimum samples required at a leaf node
    random_state=42           # Ensures reproducibility
)

# Train the model
rf_clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_clf.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest Accuracy: {accuracy * 100:.2f}%")

Random Forest Accuracy: 79.02%


with asmits modeling data

In [7]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the new modeling data
data = pd.read_csv('modeling_data.csv')

# Preview the first few rows to understand the structure
data.head()

,Unnamed: 0,Reported.Year,Property.ID..,Commercial.Property..Property.Street.Address,Building.Name,Total.SqFt,Category,Year.Built,kWh.sqft,Percentile.Rank,...,GeoLocation,Submitted.Name.of.Organization,GeoLocation_clean,lat,long,NearbyTreeSpecies,cluster,BestTreeSpecies,Category_encoded,BestTreeSpecies_encoded
0,0,2017.0,187746,"5126 E 5 ST, AUSTIN TX,","5126 E 5 ST, AUSTIN TX,",7419.0,NaN,1940,0.000,NaN,...,"5126 E 5 ST\nAUSTIN, TX\n(30.251707, -97.697313)","Pantur, Inc.","30.251707, -97.697313",30.251707,-97.697313,"Willow, Oak, Hackberry, Mesquite, Oak, Oak, Ha...",2,Pecan,27,20
1,1,2017.0,230740,"2500 RIDGEPOINT DR, AUSTIN TX, 78728","2500 RIDGEPOINT DR, AUSTIN TX, 78728",49024.0,INDUSTRIAL (W/sqft > 3),1983,15.580,39.58,...,"2500 RIDGEPOINT DR\nAUSTIN, TX 78728\n(30.3280...",Equitable Commercial Realty,"30.328069, -97.672401",30.328069,-97.672401,Oak,2,Oak,10,18
2,2,2017.0,729528,"1700 S PLEASANT VALLEY RD, AUSTIN TX, 78741","1700 S PLEASANT VALLEY RD, AUSTIN TX, 78741",101953.0,STORE\GROCERY (W/sqft > 5),2009,32.063,76.26,...,"1700 S PLEASANT VALLEY RD\nAUSTIN, TX 78741\n(...","Mimco, Inc.","30.236494, -97.721013",30.236494,-97.721013,"Crape Myrtle, Oak, Oak, Crape Myrtle, Crape My...",1,Oak,24,18
3,3,2017.0,526381,2011 W RUNDBERG LN Austin TX 78758,2011 W RUNDBERG LN Austin TX 78758,146142.0,OFFICE LG > 35000,1999,8.049,4.79,...,"2011 W RUNDBERG LN\nAustin, TX 78758\n(30.3737...",AISD,"30.373732, -97.719236",30.373732,-97.719236,"Oak, Oak, Pecan, Pecan, Sycamore, Pear, Pear, ...",2,Oak,16,18
4,4,2017.0,200001,"202 W 17 ST, AUSTIN TX, 78701","202 W 17 ST, AUSTIN TX, 78701",15350.0,OFFICE MED 10-35,1998,100.129,NaN,...,"202 W 17 ST\nAUSTIN, TX 78701\n(30.279196, -97...",Charter Communications,"30.279196, -97.740475",30.279196,-97.740475,"Pecan, Oak, Crape Myrtle, Crape Myrtle, Pecan,...",2,Oak,18,18


In [8]:
# Define features and target - update these based on the structure of modeling_data.csv
features = data[['Total.SqFt', 'Category_encoded', 'Year.Built', 'kWh.sqft', 'lat', 'long']]  # Modify if there are additional or better features
target = data['BestTreeSpecies_encoded']

# Ensure numeric data types for modeling
features = features.apply(pd.to_numeric, errors='coerce')

# Handle any missing values (fill with mean, or use other methods as needed)
features.fillna(features.mean(), inplace=True)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

# Initialize and train the Random Forest classifier
rf_clf = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)
rf_clf.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = rf_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest Accuracy with New Data: {accuracy * 100:.2f}%")

Random Forest Accuracy with New Data: 85.95%
